## Human Activity Recognition using RNN

### 0. Install the dependacy packages

In [1]:
!pip install numpy pandas tensorflow

### 1. Load the training and test data

#### 1.1 make the necessary imports

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout, LSTM, ConvLSTM2D, Flatten
from tensorflow.keras.utils import to_categorical


2022-11-28 12:09:36.173141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### 1.2 Uitility functions for loading the data

In [2]:
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [3]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

In [4]:
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [5]:
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'dataset/')
    return trainX, trainy, testX, testy


In [6]:
train_X, train_Y, test_X, test_Y = load_dataset()

### 2. Try to understand the data

In [7]:
print(train_X.shape)
print(train_Y.shape)

(7352, 128, 9)
(7352, 1)


In [8]:
print(test_X.shape)
print(test_Y.shape)

(2947, 128, 9)
(2947, 1)


In [9]:
# Features for a single subject in a single timestampe
train_X[1,1,:]

array([ 1.02238   , -0.1268078 ,  0.09935086,  0.00455008, -0.00748789,
       -0.02509841,  0.02417851,  0.00971036,  0.01614958])

In [10]:
# Possible output values
np.unique(train_Y)

array([1, 2, 3, 4, 5, 6])

### 3. Preprocessing

#### 2.1 Zero offset the output class values and one-hot encode

The output data set is in the values range 1-6. We'll convert this to the range 0-5 and one hot encode it

In [11]:
# zero-offset class values
train_Y,test_Y = train_Y -1, test_Y - 1
# one hot encode y
train_Y = to_categorical(train_Y)
test_Y = to_categorical(test_Y)

In [12]:
print(train_X.shape)
print(train_Y.shape)

(7352, 128, 9)
(7352, 6)


### 4. Train

#### 4.1 LSTM
##### Prepare the model

In [13]:
n_timesteps, n_features, n_outputs= train_X.shape[1], train_X.shape[2], train_Y.shape[1]

In [14]:
model = Sequential()
model.add(LSTM(100, input_shape = (n_timesteps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-11-28 12:09:43.465886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### Fit the model

In [15]:
epochs = 15
batch_size = 64
model.fit(train_X, train_Y, epochs = epochs, batch_size=batch_size)

Epoch 1/15
115/115 [==============================] - 7s 46ms/step - loss: 1.1609 - accuracy: 0.5039
Epoch 2/15
115/115 [==============================] - 5s 46ms/step - loss: 0.6774 - accuracy: 0.6887
Epoch 3/15
115/115 [==============================] - 5s 47ms/step - loss: 0.6010 - accuracy: 0.7418
Epoch 4/15
115/115 [==============================] - 5s 47ms/step - loss: 0.4993 - accuracy: 0.8003
Epoch 5/15
115/115 [==============================] - 6s 49ms/step - loss: 0.2815 - accuracy: 0.9017
Epoch 6/15
115/115 [==============================] - 6s 50ms/step - loss: 0.2066 - accuracy: 0.9261
Epoch 7/15
115/115 [==============================] - 6s 53ms/step - loss: 0.2730 - accuracy: 0.9057
Epoch 8/15
115/115 [==============================] - 6s 51ms/step - loss: 0.1975 - accuracy: 0.9253
Epoch 9/15
115/115 [==============================] - 6s 55ms/step - loss: 0.1708 - accuracy: 0.9368
Epoch 10/15
115/115 [==============================] - 6s 53ms/step - loss: 0.1576 - accura

##### Evaluate

In [16]:
results = model.evaluate(test_X, test_Y, batch_size = batch_size, verbose=0)

In [17]:
print("test loss, test acc:", results)

test loss, test acc: [0.29762861132621765, 0.9114353656768799]


#### 4.1 LSTM
##### Prepare the model

In [18]:
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps, n_length = 4, 32
cnnlstm_train_X = train_X.reshape((train_X.shape[0], n_steps, 1, n_length, n_features))
cnnlstm_test_X = test_X.reshape((test_X.shape[0], n_steps, 1, n_length, n_features))
cnnlstm_model = Sequential()
cnnlstm_model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
cnnlstm_model.add(Dropout(0.5))
cnnlstm_model.add(Flatten())
cnnlstm_model.add(Dense(100, activation='relu'))
cnnlstm_model.add(Dense(n_outputs, activation='softmax'))
cnnlstm_model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
epochs = 15
batch_size = 64
cnnlstm_model.fit(cnnlstm_train_X, train_Y, epochs = epochs, batch_size=batch_size)

Epoch 1/15
115/115 [==============================] - 7s 42ms/step - loss: 0.6122 - accuracy: 0.7588
Epoch 2/15
115/115 [==============================] - 5s 40ms/step - loss: 0.2214 - accuracy: 0.9159
Epoch 3/15
115/115 [==============================] - 5s 42ms/step - loss: 0.1609 - accuracy: 0.9340
Epoch 4/15
115/115 [==============================] - 5s 43ms/step - loss: 0.1376 - accuracy: 0.9415
Epoch 5/15
115/115 [==============================] - 5s 41ms/step - loss: 0.1222 - accuracy: 0.9472
Epoch 6/15
115/115 [==============================] - 5s 41ms/step - loss: 0.1236 - accuracy: 0.9493
Epoch 7/15
115/115 [==============================] - 5s 42ms/step - loss: 0.1164 - accuracy: 0.9505
Epoch 8/15
115/115 [==============================] - 5s 41ms/step - loss: 0.1066 - accuracy: 0.9531
Epoch 9/15
115/115 [==============================] - 5s 41ms/step - loss: 0.1058 - accuracy: 0.9547
Epoch 10/15
115/115 [==============================] - 5s 41ms/step - loss: 0.1017 - accura

In [20]:
results = cnnlstm_model.evaluate(cnnlstm_X, test_Y, batch_size = batch_size, verbose=0)

NameError: name 'cnnlstm_X' is not defined

In [ ]:
print("test loss, test acc:", results)